In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from utils.data_processing_tick import convert_to_tick
from utils.data_processing_tick import create_summary
from utils.calendar_utils import remove_friday
from utils.calendar_utils import extract_weekday
from utils.calendar_utils import filter_by_session_nyc
notebook_path = os.path.abspath(os.path.join(os.getcwd(), '../notebooks'))
from utils.histdata import import_pair
output_file_path = rf'{notebook_path}\data\processed'
model_output_filepath = rf'{notebook_path}\output\model'

**Get Data from histdata.com**

In [2]:
# Get intraday data from histdata.com
%debug
year = 2024
currency = "gbpusd"
import_pair(pair=currency, start=f"{year}01", end=f"{year}12", data_directory=r"data/raw")

ERROR:root:No traceback has been produced, nothing to debug.


Output()

Output()

Output()

Output()

**Split into Sessions**

In [2]:
"""
    Filters the DataFrame by the given trading session or sub-session using NYC time as a base.
   
    Trading session/sub-session times (NYC time):
    - asian: 19:00 (prev day) - 04:00
    - asian_morning: 19:00 (prev day) - 01:00
    - london: 03:00 - 12:00
    - london_morning: 03:00 - 07:00
    - london_afternoon: 07:00 - 12:00
    - ny: 08:00 - 17:00
    - ny_morning: 08:00 - 12:00
    - ny_evening: 12:00 - 17:00
   
    Parameters:
        df (DataFrame): The input time series DataFrame (with NYC timezone-aware timestamps).
        session (str): The trading session or sub-session ('asian', 'london_morning', etc.).
        output_file (str): Path to save the filtered DataFrame as a CSV file. If None, no file is saved.

    Returns:
        DataFrame: Filtered DataFrame for the session.
"""
%debug
year = "2022"
ccy = "eurusd"
session = "ny_morning"
base_price = "prev"

file_path = rf'{notebook_path}\data\raw\ASCII\M1\{ccy.lower()}\{year}\DAT_ASCII_{ccy.upper()}_M1_{year}.csv'
output_filepath = rf'{notebook_path}\data\processed\session\{year}_{ccy}_{session}.csv'

filtered_df = filter_by_session_nyc(file_path, session, output_filepath)

ERROR:root:No traceback has been produced, nothing to debug.


Filtering session: ny_morning
Filtered rows for ny_morning:
                              open     high      low    close  volume
datetime                                                             
2022-01-03 08:00:00-05:00  1.13588  1.13588  1.13553  1.13555       0
2022-01-03 08:01:00-05:00  1.13561  1.13563  1.13545  1.13557       0
2022-01-03 08:02:00-05:00  1.13560  1.13565  1.13535  1.13535       0
2022-01-03 08:03:00-05:00  1.13535  1.13545  1.13535  1.13540       0
2022-01-03 08:04:00-05:00  1.13539  1.13546  1.13534  1.13543       0
...                            ...      ...      ...      ...     ...
2022-12-30 11:55:00-05:00  1.06880  1.06884  1.06861  1.06865       0
2022-12-30 11:56:00-05:00  1.06864  1.06895  1.06847  1.06895       0
2022-12-30 11:57:00-05:00  1.06895  1.06925  1.06895  1.06925       0
2022-12-30 11:58:00-05:00  1.06925  1.06971  1.06925  1.06953       0
2022-12-30 11:59:00-05:00  1.06955  1.06969  1.06947  1.06967       0

[62370 rows x 5 columns]
Filt

In [13]:
%debug
year = "2024"
ccy = "usdjpy"
sessions = ["ny", "ny_morning", "ny_evening", "london", "london_morning", "london_afternoon", "asian", "asian_morning"]

for session in sessions:
    file_path = rf'{notebook_path}\data\raw\ASCII\M1\{ccy.lower()}\{year}\DAT_ASCII_{ccy.upper()}_M1_{year}.csv'
    output_filepath = rf'{notebook_path}\data\processed\session\{year}_{ccy}_{session}.csv'

    filtered_df = filter_by_session_nyc(file_path, session, output_filepath)

ERROR:root:No traceback has been produced, nothing to debug.


Filtering session: ny
Filtered rows for ny:
                              open     high      low    close  volume
datetime                                                             
2024-01-02 08:00:00-05:00  142.114  142.139  142.089  142.094       0
2024-01-02 08:01:00-05:00  142.095  142.126  142.094  142.126       0
2024-01-02 08:02:00-05:00  142.126  142.133  142.104  142.115       0
2024-01-02 08:03:00-05:00  142.113  142.154  142.113  142.122       0
2024-01-02 08:04:00-05:00  142.123  142.133  142.114  142.123       0
...                            ...      ...      ...      ...     ...
2024-12-31 16:54:00-05:00  157.237  157.237  157.216  157.224       0
2024-12-31 16:55:00-05:00  157.224  157.226  157.215  157.219       0
2024-12-31 16:56:00-05:00  157.219  157.229  157.196  157.211       0
2024-12-31 16:57:00-05:00  157.216  157.222  157.206  157.208       0
2024-12-31 16:58:00-05:00  157.209  157.212  157.193  157.197       0

[139152 rows x 5 columns]
Filtered data saved

**Create Renko Brick Data**

In [2]:
%debug

year = "2024"
ccy = "usdjpy"
session = "asian"
brick_size = 0.1 # 0.001 for EURUSD/EURGBP/GBPUSD, 0.1 for USDJPY
use_first_price_as_base=True # True: use first price as base, False: use previous price as base

csv_filepath = rf'{notebook_path}\data\processed\session\timestamp_data\{year}_{ccy}_{session}.csv'
output_file_path = rf'{notebook_path}\data\processed\session\timestamp_data\renko'

if (use_first_price_as_base):
    output_filename = f'{year}_{ccy.lower()}_{session}_tick_first_price_as_base.csv'
else:
    output_filename = f'{year}_{ccy.lower()}_{session}_tick_prev_price_as_base.csv'

convert_to_tick(csv_filepath, output_file_path, output_filename, brick_size, use_first_price_as_base) 

ERROR:root:No traceback has been produced, nothing to debug.


datetime64[ns, America/New_York]
0   2024-01-01 19:00:00-05:00
1   2024-01-01 19:01:00-05:00
2   2024-01-01 19:02:00-05:00
3   2024-01-01 19:03:00-05:00
4   2024-01-01 19:04:00-05:00
Name: datetime, dtype: datetime64[ns, America/New_York]
0
Tick-like data has been saved to tick_like_data.csv
   tick_number                  datetime  renko_price  actual_openprice  \
0            1 2024-01-01 19:17:00-05:00       141.21           141.216   
1            2 2024-01-01 19:24:00-05:00       141.31           141.319   
2            3 2024-01-01 19:54:00-05:00       141.21           141.206   
3            4 2024-01-01 20:29:00-05:00       141.31           141.323   
4            5 2024-01-01 20:35:00-05:00       141.41           141.425   

   ticks_moved directions day_of_week  
0            1         +1      Monday  
1            1         +1      Monday  
2            1         -1      Monday  
3            1         +1      Monday  
4            1         +1      Monday  


,tick_number,datetime,renko_price,actual_openprice,ticks_moved,directions,day_of_week
0,1,2024-01-01 19:17:00-05:00,141.21,141.216,1,+1,Monday
1,2,2024-01-01 19:24:00-05:00,141.31,141.319,1,+1,Monday
2,3,2024-01-01 19:54:00-05:00,141.21,141.206,1,-1,Monday
3,4,2024-01-01 20:29:00-05:00,141.31,141.323,1,+1,Monday
4,5,2024-01-01 20:35:00-05:00,141.41,141.425,1,+1,Monday
...,...,...,...,...,...,...,...
5769,5770,2024-12-31 01:56:00-05:00,156.31,156.311,1,+1,Tuesday
5770,5771,2024-12-31 02:28:00-05:00,156.21,156.202,1,-1,Tuesday
5771,5772,2024-12-31 02:36:00-05:00,156.11,156.080,1,-1,Tuesday
5772,5773,2024-12-31 03:01:00-05:00,156.21,156.213,1,+1,Tuesday


In [17]:
year = "2024"
ccy = "gbpusd"
brick_size = 0.001 # 0.001 for EURUSD/EURGBP/GBPUSD, 0.1 for USDJPY
use_first_price_as_base=True # True: use first price as base, False: use previous price as base
sessions = ["ny", "ny_morning", "ny_evening", "london", "london_morning", "london_afternoon", "asian", "asian_morning"]

for session in sessions:
    csv_filepath = rf'{notebook_path}\data\processed\session\timestamp_data\{year}_{ccy}_{session}.csv'
    output_file_path = rf'{notebook_path}\data\processed\session\timestamp_data\renko'

    if (use_first_price_as_base):
        output_filename = f'{year}_{ccy.lower()}_{session}_tick_first_price_as_base.csv'
    else:
        output_filename = f'{year}_{ccy.lower()}_{session}_tick_prev_price_as_base.csv'

    convert_to_tick(csv_filepath, output_file_path, output_filename, brick_size, use_first_price_as_base) 

datetime64[ns, America/New_York]
0   2024-01-02 08:00:00-05:00
1   2024-01-02 08:01:00-05:00
2   2024-01-02 08:02:00-05:00
3   2024-01-02 08:03:00-05:00
4   2024-01-02 08:04:00-05:00
Name: datetime, dtype: datetime64[ns, America/New_York]
0
Tick-like data has been saved to tick_like_data.csv
   tick_number                  datetime  renko_price  actual_openprice  \
0            1 2024-01-02 08:50:00-05:00      1.26334           1.26320   
1            2 2024-01-02 09:12:00-05:00      1.26234           1.26227   
2            3 2024-01-02 09:23:00-05:00      1.26334           1.26352   
3            4 2024-01-02 09:35:00-05:00      1.26434           1.26438   
4            5 2024-01-02 09:44:00-05:00      1.26334           1.26331   

   ticks_moved directions day_of_week  
0            1         -1     Tuesday  
1            1         -1     Tuesday  
2            1         +1     Tuesday  
3            1         +1     Tuesday  
4            1         -1     Tuesday  
datetime64[ns, A

**Create Summary Table**

In [2]:
model_output_filepath = rf'{notebook_path}\output\model\random_forest'
result_file = "random_forest_results"
create_summary(model_output_filepath, result_file)

Files found: ['c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_morning_backtesting.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_morning_random_forest_dataframe.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_morning_random_forest_results.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_random_forest_dataframe.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_asian_random_forest_results.csv', 'c:\\Users\\mmori\\Documents\\fx_strategy_project\\notebooks\\output\\model\\random_forest\\2022_eurgbp_london_afternoon_backtesting.csv', 'c:\\Users\